# Week 10: Tutorials (MNE Python Toolkit)

Source: [MNE-Python](https://mne.tools/stable/index.html)

# 1. Introduction

These tutorials cover the basic EEG/MEG pipeline for event-related analysis, introduce the [mne.Info](https://mne.tools/stable/generated/mne.Info.html#mne.Info), [events](https://mne.tools/stable/documentation/glossary.html#term-events), and [mne.Annotations](https://mne.tools/stable/generated/mne.Annotations.html#mne.Annotations) data structures, discuss how sensor locations are handled, and introduce some of the configuration options available.

## 1.1 Overview of MEG/EEG analysis with MNE-Python

This tutorial covers the basic EEG/MEG pipeline for event-related analysis: loading data, epoching, averaging, plotting, and estimating cortical activity from sensor data. It introduces the core MNE-Python data structures [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw), [Epochs](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs), [Evoked](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked), and [SourceEstimate](https://mne.tools/stable/generated/mne.SourceEstimate.html#mne.SourceEstimate), and covers a lot of ground fairly quickly (at the expense of depth). Subsequent tutorials address each of these topics in greater detail.

We begin by importing the necessary Python modules:

In [ ]:
# !pip install mne
# !pip install ipywidgets
# !pip install pyvistaqt
# !pip install nibabel
# !pip install h5io
# !pip install pyQt5

In [ ]:
import os
import mne
import PyQt5
import tempfile
import numpy as np
import scipy.ndimage
from pathlib import Path
import matplotlib.pyplot as plt

# %matplotlib qt5

### Loading data

MNE-Python data structures are based around the FIF file format from Neuromag, but there are reader functions for [a wide variety of other data formats](https://mne.tools/stable/documentation/implementation.html#data-formats). MNE-Python also has interfaces to a variety of [publicly available datasets](https://mne.tools/stable/documentation/datasets.html#datasets), which MNE-Python can download and manage for you.

We’ll start this tutorial by loading one of the example datasets (called “[Sample](https://mne.tools/stable/documentation/datasets.html#sample-dataset)”), which contains EEG and MEG data from one subject performing an audiovisual experiment, along with structural MRI scans for that subject. The [mne.datasets.sample.data_path](https://mne.tools/stable/generated/mne.datasets.sample.data_path.html#mne.datasets.sample.data_path) function will automatically download the dataset if it isn’t found in one of the expected locations, then return the directory path to the dataset (see the documentation of [data_path](https://mne.tools/stable/generated/mne.datasets.sample.data_path.html#mne.datasets.sample.data_path) for a list of places it checks before downloading). Note also that for this tutorial to run smoothly on our servers, we’re using a filtered and downsampled version of the data (`sample_audvis_filt-0-40_raw.fif`), but an unfiltered version (`sample_audvis_raw.fif`) is also included in the sample dataset and could be substituted here when running the tutorial locally.

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = (
    sample_data_folder / "MEG" / "sample" / "sample_audvis_filt-0-40_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)

By default, [read_raw_fif](https://mne.tools/stable/generated/mne.io.read_raw_fif.html#mne.io.read_raw_fif) displays some information about the file it’s loading; for example, here it tells us that there are four “projection items” in the file along with the recorded data; those are [SSP projectors](https://mne.tools/stable/documentation/glossary.html#term-projector) calculated to remove environmental noise from the MEG signals, plus a projector to mean-reference the EEG channels; these are discussed in the tutorial calculated to remove environmental noise from the MEG signals, plus a projector to mean-reference the EEG channels; these are discussed in the tutorial [Background on projectors and projections](https://mne.tools/stable/auto_tutorials/preprocessing/45_projectors_background.html#tut-projectors-background). In addition to the information displayed during loading, you can get a glimpse of the basic details of a [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw) object by printing it; even more is available by printing its `info` attribute (a [dictionary-like object](https://mne.tools/stable/generated/mne.Info.html#mne.Info) that is preserved across [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw), [Epochs](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs), and [Evoked](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked) objects). The `info` data structure keeps track of channel locations, applied filters, projectors, etc. Notice especially the `chs` entry, showing that MNE-Python detects different sensor types and handles each appropriately. See [The Info data structure](https://mne.tools/stable/auto_tutorials/intro/30_info.html#tut-info-class) for more on the [Info](https://mne.tools/stable/generated/mne.Info.html#mne.Info) class.

In [ ]:
print(raw)
print(raw.info)

`Raw` objects also have several built-in plotting methods; here we show the power spectral density (PSD) for each sensor type with `compute_psd`, as well as a plot of the raw sensor traces with `plot`. In the PSD plot, we’ll only plot frequencies below 50 Hz (since our data are low-pass filtered at 40 Hz). In interactive Python sessions, `plot` is interactive and allows scrolling, scaling, bad channel marking, annotations, projector toggling, etc.

In [ ]:
raw.compute_psd(fmax=50).plot(picks="data", exclude="bads")
raw.plot(duration=5, n_channels=30)

### Preprocessing

MNE-Python supports a variety of preprocessing approaches and techniques (maxwell filtering, signal-space projection, independent components analysis, filtering, downsampling, etc); see the full list of capabilities in the `mne.preprocessing` and `mne.filter` submodules. Here we’ll clean up our data by performing independent components analysis (`ICA`); for brevity we’ll skip the steps that helped us determined which components best capture the artifacts (see `Repairing artifacts with ICA` for a detailed walk-through of that process).

In [ ]:
# set up and fit the ICA
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)
ica.exclude = [1, 2]  # details on how we picked these are omitted here
ica.plot_properties(raw, picks=ica.exclude)

Once we’re confident about which component(s) we want to remove, we pass them as the `exclude` parameter and then apply the ICA to the raw signal. The `apply` method requires the raw data to be loaded into memory (by default it’s only read from disk as-needed), so we’ll use `load_data` first. We’ll also make a copy of the `Raw` object so we can compare the signal before and after artifact removal side-by-side:

In [ ]:
orig_raw = raw.copy()
raw.load_data()
ica.apply(raw)

# show some frontal channels to clearly illustrate the artifact removal
chs = [
    "MEG 0111",
    "MEG 0121",
    "MEG 0131",
    "MEG 0211",
    "MEG 0221",
    "MEG 0231",
    "MEG 0311",
    "MEG 0321",
    "MEG 0331",
    "MEG 1511",
    "MEG 1521",
    "MEG 1531",
    "EEG 001",
    "EEG 002",
    "EEG 003",
    "EEG 004",
    "EEG 005",
    "EEG 006",
    "EEG 007",
    "EEG 008",
]
chan_idxs = [raw.ch_names.index(ch) for ch in chs]
orig_raw.plot(order=chan_idxs, start=12, duration=4)
raw.plot(order=chan_idxs, start=12, duration=4)

### Detecting experimental events

The sample dataset includes several `“STIM” channels` that recorded electrical signals sent from the stimulus delivery computer (as brief DC shifts / squarewave pulses). These pulses (often called “triggers”) are used in this dataset to mark experimental events: stimulus onset, stimulus type, and participant response (button press). The individual STIM channels are combined onto a single channel, in such a way that voltage levels on that channel can be unambiguously decoded as a particular event type. On older Neuromag systems (such as that used to record the sample data) this summation channel was called `STI 014`, so we can pass that channel name to the `mne.find_events` function to recover the timing and identity of the stimulus events.

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")
print(events[:5])  # show the first 5

The resulting events array is an ordinary 3-column `NumPy array`, with sample number in the first column and integer event ID in the last column; the middle column is usually ignored. Rather than keeping track of integer event IDs, we can provide an event dictionary that maps the integer IDs to experimental conditions or events. In this dataset, the mapping looks like this:

Event ID | Condition
-------- | ---------
1        | auditory stimulus (tone) to the left ear
2        | auditory stimulus (tone) to the right ear
3        | visual stimulus (checkerboard) to the left visual field
4        | visual stimulus (checkerboard) to the right visual field
5        | smiley face (catch trial)
32       | subject button press

In [ ]:
event_dict = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
    "smiley": 5,
    "buttonpress": 32,
}

Event dictionaries like this one are used when extracting epochs from continuous data; the `/` character in the dictionary keys allows pooling across conditions by requesting partial condition descriptors (i.e., requesting `'auditory'` will select all epochs with Event IDs 1 and 2; requesting `'left'` will select all epochs with Event IDs 1 and 3). An example of this is shown in the next section. There is also a convenient `plot_events` function for visualizing the distribution of events across the duration of the recording (to make sure event detection worked as expected). Here we’ll also make use of the `Info` attribute to get the sampling frequency of the recording (so our x-axis will be in seconds instead of in samples).

In [ ]:
fig = mne.viz.plot_events(
    events, event_id=event_dict, sfreq=raw.info["sfreq"], first_samp=raw.first_samp
)

For paradigms that are not event-related (e.g., analysis of resting-state data), you can extract regularly spaced (possibly overlapping) spans of data by creating events using `mne.make_fixed_length_events` and then proceeding with epoching as described in the next section.

### Epoching continuous data

The `Raw` object and the events array are the bare minimum needed to create an `Epochs` object, which we create with the `Epochs` class constructor. Here we’ll also specify some data quality constraints: we’ll reject any epoch where peak-to-peak signal amplitude is beyond reasonable limits for that channel type. This is done with a rejection dictionary; you may include or omit thresholds for any of the channel types present in your data. The values given here are reasonable for this particular dataset, but may need to be adapted for different hardware or recording conditions. For a more automated approach, consider using the `autoreject package`.

In [ ]:
reject_criteria = dict(
    mag=4000e-15,  # 4000 fT
    grad=4000e-13,  # 4000 fT/cm
    eeg=150e-6,  # 150 µV
    eog=250e-6,
)  # 250 µV

We’ll also pass the event dictionary as the `event_id` parameter (so we can work with easy-to-pool event labels instead of the integer event IDs), and specify `tmin` and `tmax` (the time relative to each event at which to start and end each epoch). As mentioned above, by default `Raw` and `Epochs` data aren’t loaded into memory (they’re accessed from disk only when needed), but here we’ll force loading into memory using the `preload=True` parameter so that we can see the results of the rejection criteria being applied:

In [ ]:
epochs = mne.Epochs(
    raw,
    events,
    event_id=event_dict,
    tmin=-0.2,
    tmax=0.5,
    reject=reject_criteria,
    preload=True,
)

Next we’ll pool across left/right stimulus presentations so we can compare auditory versus visual responses. To avoid biasing our signals to the left or right, we’ll use `equalize_event_counts` first to randomly sample epochs from each condition to match the number of epochs present in the condition with the fewest good epochs.

In [ ]:
conds_we_care_about = ["auditory/left", "auditory/right", "visual/left", "visual/right"]
epochs.equalize_event_counts(conds_we_care_about)  # this operates in-place
aud_epochs = epochs["auditory"]
vis_epochs = epochs["visual"]
del raw, epochs  # free up memory

Like `Raw` objects, `Epochs` objects also have a number of built-in plotting methods. One is `plot_image`, which shows each epoch as one row of an image map, with color representing signal magnitude; the average evoked response and the sensor location are shown below the image:

In [ ]:
aud_epochs.plot_image(picks=["MEG 1332", "EEG 021"])

### Time-frequency analysis

The `mne.time_frequency` submodule provides implementations of several algorithms to compute time-frequency representations, power spectral density, and cross-spectral density. Here, for example, we’ll compute for the auditory epochs the induced power at different frequencies and times, using Morlet wavelets. On this dataset the result is not especially informative (it just shows the evoked “auditory N100” response); see [here](https://mne.tools/stable/auto_tutorials/time-freq/20_sensors_time_frequency.html#inter-trial-coherence) for a more extended example on a dataset with richer frequency content.

In [ ]:
frequencies = np.arange(7, 30, 3)
power = mne.time_frequency.tfr_morlet(
    aud_epochs, n_cycles=2, return_itc=False, freqs=frequencies, decim=3
)
power.plot(["MEG 1332"])

### Estimating evoked responses

Now that we have our conditions in `aud_epochs` and `vis_epochs`, we can get an estimate of evoked responses to auditory versus visual stimuli by averaging together the epochs in each condition. This is as simple as calling the `average` method on the `Epochs` object, and then using a function from the `mne.viz` module to compare the global field power for each sensor type of the two `Evoked` objects:

In [ ]:
aud_evoked = aud_epochs.average()
vis_evoked = vis_epochs.average()

mne.viz.plot_compare_evokeds(
    dict(auditory=aud_evoked, visual=vis_evoked),
    legend="upper left",
    show_sensors="upper right",
)

We can also get a more detailed view of each `Evoked` object using other plotting methods such as `plot_joint` or `plot_topomap`. Here we’ll examine just the EEG channels, and see the classic auditory evoked N100-P200 pattern over dorso-frontal electrodes, then plot scalp topographies at some additional arbitrary times:

In [ ]:
aud_evoked.plot_joint(picks="eeg")
aud_evoked.plot_topomap(times=[0.0, 0.08, 0.1, 0.12, 0.2], ch_type="eeg")

Evoked objects can also be combined to show contrasts between conditions, using the `mne.combine_evoked` function. A simple difference can be generated by passing `weights=[1, -1]`. We’ll then plot the difference wave at each sensor using `plot_topo`:

In [ ]:
evoked_diff = mne.combine_evoked([aud_evoked, vis_evoked], weights=[1, -1])
evoked_diff.pick(picks="mag").plot_topo(color="r", legend=False)

### Inverse modeling

Finally, we can estimate the origins of the evoked activity by projecting the sensor data into this subject’s `source space` (a set of points either on the cortical surface or within the cortical volume of that subject, as estimated by structural MRI scans). MNE-Python supports lots of ways of doing this (dynamic statistical parametric mapping, dipole fitting, beamformers, etc.); here we’ll use minimum-norm estimation (MNE) to generate a continuous map of activation constrained to the cortical surface. MNE uses a linear `inverse operator` to project EEG+MEG sensor measurements into the source space. The inverse operator is computed from the `forward solution` for this subject and an estimate of `the covariance of sensor measurements`. For this tutorial we’ll skip those computational steps and load a pre-computed inverse operator from disk (it’s included with the `sample data`). Because this “inverse problem” is underdetermined (there is no unique solution), here we further constrain the solution by providing a regularization parameter specifying the relative smoothness of the current estimates in terms of a signal-to-noise ratio (where “noise” here is akin to baseline activity level across all of cortex).

In [ ]:
# load inverse operator
inverse_operator_file = (
    sample_data_folder / "MEG" / "sample" / "sample_audvis-meg-oct-6-meg-inv.fif"
)
inv_operator = mne.minimum_norm.read_inverse_operator(inverse_operator_file)
# set signal-to-noise ratio (SNR) to compute regularization parameter (λ²)
snr = 3.0
lambda2 = 1.0 / snr**2
# generate the source time course (STC)
stc = mne.minimum_norm.apply_inverse(
    vis_evoked, inv_operator, lambda2=lambda2, method="MNE"
)  # or dSPM, sLORETA, eLORETA

Finally, in order to plot the source estimate on the subject’s cortical surface we’ll also need the path to the sample subject’s structural MRI files (the `subjects_dir`):

In [ ]:
# path to subjects' MRI files
subjects_dir = sample_data_folder / "subjects"
# plot the STC
stc.plot(
    initial_time=0.1, hemi="split", views=["lat", "med"], subjects_dir=subjects_dir
)

## 1.2 Modifying data in-place

Many of MNE-Python’s data objects (`Raw`, `Epochs`, `Evoked`, etc) have methods that modify the data in-place (either optionally or obligatorily). This can be advantageous when working with large datasets because it reduces the amount of computer memory needed to perform the computations. However, it can lead to unexpected results if you’re not aware that it’s happening. This tutorial provides a few examples of in-place processing, and how and when to avoid it.

As usual we’ll start by importing the modules we need and loading some `example data`:

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
# the preload flag loads the data into memory now
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw.crop(tmax=10.0)  # raw.crop() always happens in-place

### Signal processing

Most MNE-Python data objects have built-in methods for filtering, including high-, low-, and band-pass filters (`filter`), band-stop filters (`notch_filter`), Hilbert transforms (`apply_hilbert`), and even arbitrary or user-defined functions (`apply_function`). These typically always modify data in-place, so if we want to preserve the unprocessed data for comparison, we must first make a copy of it. For example:

In [ ]:
original_raw = raw.copy()
raw.apply_hilbert()
print(
    f"original data type was {original_raw.get_data().dtype}, after "
    f"apply_hilbert the data type changed to {raw.get_data().dtype}."
)

### Channel picking

Another group of methods where data is modified in-place are the channel-picking methods. For example:

In [ ]:
print(f'original data had {original_raw.info["nchan"]} channels.')
original_raw.pick("eeg")  # selects only the EEG channels
print(f'after picking, it has {original_raw.info["nchan"]} channels.')

Note also that when picking only EEG channels, projectors that affected only the magnetometers were dropped, since there are no longer any magnetometer channels.

### The `copy` parameter

Above we saw an example of using the `copy` method to facilitate comparing data before and after processing. This is not needed when using certain MNE-Python functions, because they have a function parameter where you can specify `copy=True` (return a modified copy of the data) or `copy=False` (operate in-place). For example, `mne.set_eeg_reference` is one such function; notice that here we plot `original_raw` after the rereferencing has been done, but `original_raw` is unaffected because we specified `copy=True`:

In [ ]:
rereferenced_raw, ref_data = mne.set_eeg_reference(original_raw, ["EEG 003"], copy=True)
fig_orig = original_raw.plot()
fig_reref = rereferenced_raw.plot()

Another example is the picking function `mne.pick_info`, which operates on `mne.Info` dictionaries rather than on data objects. See [The Info data structure](https://mne.tools/stable/auto_tutorials/intro/30_info.html#tut-info-class) for details.

## 1.3 Parsing events from raw data

This tutorial describes how to read experimental events from raw recordings, and how to convert between the two different representations of events within MNE-Python (Events arrays and Annotations objects).

In the `introductory tutorial` we saw an
example of reading experimental events from a :term:`"STIM" channel`; here we'll discuss `events` and `annotations` more broadly, give more detailed information about reading from STIM channels, and give an example of reading events that are in a marker file or included in the data file as an embedded array. The tutorials `Working with events` and `Annotation continuous data` discuss how to plot, combine, load, save, and export `events` and `mne.Annotations` (respectively), and the latter tutorial also covers interactive annotation of `mne.io.Raw` objects.

We'll begin by loading the Python modules we need, and loading the same
`example data` we used in the `introductory tutorial`, but to save memory we'll crop the `mne.io.Raw` object
to just 60 seconds before loading it into RAM:

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(tmax=60).load_data()

### The Events and Annotations data structures

Generally speaking, both the Events and `mne.Annotations` data structures serve the same purpose: they provide a mapping between times during an EEG/MEG recording and a description of what happened at those times. In other words, they associate a *when* with a *what*. The main differences are:

1. **Units**: the Events data structure represents the *when* in terms of
   samples, whereas the `mne.Annotations` data structure represents
   the *when* in seconds.
2. **Limits on the description**: the Events data structure represents the
   *what* as an integer "Event ID" code, whereas the `mne.Annotations` data
   structure represents the *what* as a string.
3. **How duration is encoded**: Events in an Event array do not have a
   duration (though it is possible to represent duration with pairs of
   onset/offset events within an Events array), whereas each element of an
   `mne.Annotations` object necessarily includes a duration (though
   the duration can be zero if an instantaneous event is desired).
4. **Internal representation**: Events are stored as an ordinary `NumPy array`, whereas `mne.Annotations` is
   a `list`-like class defined in MNE-Python.

### What is a STIM channel?

A `stim channel` (short for “stimulus channel”) is a channel that does not receive signals from an EEG, MEG, or other sensor. Instead, STIM channels record voltages (usually short, rectangular DC pulses of fixed magnitudes sent from the experiment-controlling computer) that are time-locked to experimental events, such as the onset of a stimulus or a button-press response by the subject (those pulses are sometimes called `TTL` pulses, event pulses, trigger signals, or just “triggers”). In other cases, these pulses may not be strictly time-locked to an experimental event, but instead may occur in between trials to indicate the type of stimulus (or experimental condition) that is about to occur on the upcoming trial.

The DC pulses may be all on one STIM channel (in which case different experimental events or trial types are encoded as different voltage magnitudes), or they may be spread across several channels, in which case the channel(s) on which the pulse(s) occur can be used to encode different events or conditions. Even on systems with multiple STIM channels, there is often one channel that records a weighted sum of the other STIM channels, in such a way that voltage levels on that channel can be unambiguously decoded as particular event types. On older Neuromag systems (such as that used to record the sample data) this “summation channel” was typically `STI 014`; on newer systems it is more commonly `STI101`. You can see the STIM channels in the raw data file here:

In [ ]:
raw.copy().pick(picks="stim").plot(start=3, duration=6)

You can see that `STI 014` (the summation channel) contains pulses of different magnitudes whereas pulses on other channels have consistent magnitudes. You can also see that every time there is a pulse on one of the other STIM channels, there is a corresponding pulse on `STI 014`.

### Converting a STIM channel signal to an Events array

If your data has events recorded on a STIM channel, you can convert them into an events array using `find_events`. The sample number of the onset (or offset) of each pulse is recorded as the event time, the pulse magnitudes are converted into integers, and these pairs of sample numbers plus integer codes are stored in `NumPy arrays` (usually called “the events array” or just “the events”). In its simplest form, the function requires only the `Raw` object, and the name of the channel(s) from which to read events:

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")
print(events[:5])  # show the first 5

If you don’t provide the name of a STIM channel, `find_events` will first look for MNE-Python `config variables` for variables `MNE_STIM_CHANNEL`, `MNE_STIM_CHANNEL_1`, etc. If those are not found, channels `STI 014` and `STI101` are tried, followed by the first channel with type “STIM” present in `raw.ch_names`. If you regularly work with data from several different MEG systems with different STIM channel names, setting the `MNE_STIM_CHANNEL` config variable may not be very useful, but for researchers whose data is all from a single system it can be a time-saver to configure that variable once and then forget about it.

`find_events` has several options, including options for aligning events to the onset or offset of the STIM channel pulses, setting the minimum pulse duration, and handling of consecutive pulses (with no return to zero between them). For example, you can effectively encode event duration by passing `output='step'` to `find_events`; see the documentation of `find_events` for details. More information on working with events arrays (including how to plot, combine, load, and save event arrays) can be found in the tutorial `Working with events`.

### Reading embedded events as Annotations

Some EEG/MEG systems generate files where events are stored in a separate data array rather than as pulses on one or more STIM channels. For example, the EEGLAB format stores events as a collection of arrays in the `.set` file. When reading those files, MNE-Python will automatically convert the stored events into an `Annotations` object and store it as the `annotations` attribute of the Raw` object:

In [ ]:
testing_data_folder = mne.datasets.testing.data_path()
eeglab_raw_file = testing_data_folder / "EEGLAB" / "test_raw.set"
eeglab_raw = mne.io.read_raw_eeglab(eeglab_raw_file)
print(eeglab_raw.annotations)

The core data within an `Annotations` object is accessible through three of its attributes: `onset`, `duration`, and `description`. Here we can see that there were 154 events stored in the EEGLAB file, they all had a duration of zero seconds, there were two different types of events, and the first event occurred about 1 second after the recording began:

In [ ]:
print(len(eeglab_raw.annotations))
print(set(eeglab_raw.annotations.duration))
print(set(eeglab_raw.annotations.description))
print(eeglab_raw.annotations.onset[0])

More information on working with `Annotations` objects, including how to add annotations to `Raw` objects interactively, and how to plot, concatenate, load, save, and export `Annotations` objects can be found in the tutorial `Annotating continuous data`.

### Converting between Events arrays and Annotations objects

Once your experimental events are read into MNE-Python (as either an Events array or an `Annotations` object), you can easily convert between the two formats as needed. You might do this because, e.g., an Events array is needed for epoching continuous data, or because you want to take advantage of the “annotation-aware” capability of some functions, which automatically omit spans of data if they overlap with certain annotations.

To convert an `Annotations` object to an Events array, use the function `mne.events_from_annotations` on the `Raw` file containing the annotations. This function will assign an integer Event ID to each unique element of `raw.annotations.description`, and will return the mapping of descriptions to integer Event IDs along with the derived Event array. By default, one event will be created at the onset of each annotation; this can be modified via the chunk_duration parameter of `events_from_annotations` to create equally spaced events within each annotation span (see `Making multiple events per annotation`, below, or see `Making equally-spaced Events arrays` for direct creation of an Events array of equally-spaced events).

In [ ]:
events_from_annot, event_dict = mne.events_from_annotations(eeglab_raw)
print(event_dict)
print(events_from_annot[:5])

If you want to control which integers are mapped to each unique description value, you can pass a `dict` specifying the mapping as the `event_id` parameter of `events_from_annotations`; this `dict` will be returned unmodified as the `event_dict`.

Note that this `event_dict` can be used when creating `Epochs` from `Raw` objects, as demonstrated in the tutorial `The Epochs data structure: discontinuous data`.

In [ ]:
custom_mapping = {"rt": 77, "square": 42}
(events_from_annot, event_dict) = mne.events_from_annotations(
    eeglab_raw, event_id=custom_mapping
)
print(event_dict)
print(events_from_annot[:5])

To make the opposite conversion (from an Events array to an `Annotations` object), you can create a mapping from integer Event ID to string descriptions, use `annotations_from_events` to construct the `Annotations` object, and call the `set_annotations` method to add the annotations to the `Raw` object.

Because the `sample data` was recorded on a Neuromag system (where sample numbering starts when the acquisition system is initiated, not when the recording is initiated), we also need to pass in the `orig_time` parameter so that the onsets are properly aligned relative to the start of recording:

In [ ]:
mapping = {
    1: "auditory/left",
    2: "auditory/right",
    3: "visual/left",
    4: "visual/right",
    5: "smiley",
    32: "buttonpress",
}
annot_from_events = mne.annotations_from_events(
    events=events,
    event_desc=mapping,
    sfreq=raw.info["sfreq"],
    orig_time=raw.info["meas_date"],
)
raw.set_annotations(annot_from_events)

Now, the annotations will appear automatically when plotting the raw data, and will be color-coded by their label value:

In [ ]:
raw.plot(start=5, duration=5)

### Making multiple events per annotation

As mentioned above, you can generate equally-spaced events from an `Annotations` object using the `chunk_duration` parameter of `events_from_annotations`. For example, suppose we have an annotation in our `Raw` object indicating when the subject was in REM sleep, and we want to perform a resting-state analysis on those spans of data. We can create an Events array with a series of equally-spaced events within each “REM” span, and then use those events to generate (potentially overlapping) epochs that we can analyze further.

In [ ]:
# create the REM annotations
rem_annot = mne.Annotations(onset=[5, 41], duration=[16, 11], description=["REM"] * 2)
raw.set_annotations(rem_annot)
(rem_events, rem_event_dict) = mne.events_from_annotations(raw, chunk_duration=1.5)

Now we can check that our events indeed fall in the ranges 5-21 seconds and 41-52 seconds, and are ~1.5 seconds apart (modulo some jitter due to the sampling frequency). Here are the event times rounded to the nearest millisecond:

In [ ]:
print(np.round((rem_events[:, 0] - raw.first_samp) / raw.info["sfreq"], 3))

Other examples of resting-state analysis can be found in the online documentation for [make_fixed_length_events](https://mne.tools/stable/generated/mne.make_fixed_length_events.html#mne.make_fixed_length_events).

## 1.4 The Info data structure

This tutorial describes the `mne.Info` data structure, which keeps track of various recording details, and is attached to `Raw`, `Epochs`, and `Evoked` objects.

We will begin by loading the Python modules we need, and loading the `same example` data we used in the `introductory tutorial`:

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = (
    sample_data_folder / "MEG" / "sample" / "sample_audvis_filt-0-40_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)

As seen in the `introductory tutorial`, when a `Raw` object is loaded, an `Info` object is created automatically, and stored in the `raw.info` attribute:

In [ ]:
print(raw.info)

However, it is not strictly necessary to load the `Raw` object in order to view or edit the `Info` object; you can extract all the relevant information into a stand-alone `Info` object using `mne.io.read_info()`:

In [ ]:
info = mne.io.read_info(sample_data_raw_file)
print(info)

As you can see, the `Info` object keeps track of a lot of information about:

- the recording system (gantry angle, HPI details, sensor digitizations, channel names, …)

- the experiment (project name and ID, subject information, recording date, experimenter name or ID, …)

- the data (sampling frequency, applied filter frequencies, bad channels, projectors, …)

The complete list of fields is given in `the API documentation`.

### Querying the `Info` object

The fields in a `Info` object act like Python `dictionary` keys, using square brackets and strings to access the contents of a field:

In [ ]:
print(info.keys())
print()  # insert a blank line
print(info["ch_names"])

Most of the fields contain `int`, `float`, or `list` data, but the `chs` field bears special mention: it contains a list of dictionaries (one `dict` per channel) containing everything there is to know about a channel other than the data it recorded. Normally it is not necessary to dig into the details of the `chs` field — various MNE-Python functions can extract the information more cleanly than iterating over the list of dicts yourself — but it can be helpful to know what is in there. Here we show the keys for the first channel’s `dict`:

In [ ]:
print(info["chs"][0].keys())

### Obtaining subsets of channels

It is often useful to convert between channel names and the integer indices identifying rows of the data array where those channels’ measurements are stored. The `Info` object is useful for this task; two convenience functions that rely on the `mne.Info` object for picking channels are `mne.pick_channels()` and `mne.pick_types()`. `pick_channels()` minimally takes a list of all channel names and a list of channel names to include; it is also possible to provide an empty list to include and specify which channels to `exclude` instead:

In [ ]:
print(mne.pick_channels(info["ch_names"], include=["MEG 0312", "EEG 005"]))

print(mne.pick_channels(info["ch_names"], include=[], exclude=["MEG 0312", "EEG 005"]))

`pick_types()` works differently, since channel type cannot always be reliably determined from channel name alone. Consequently, `pick_types()` needs an `Info` object instead of just a list of channel names, and has boolean keyword arguments for each channel type. Default behavior is to pick only MEG channels (and MEG reference channels if present) and exclude any channels already marked as “bad” in the `bads` field of the `Info` object. Therefore, to get all and only the EEG channel indices (including the “bad” EEG channels) we must pass `meg=False` and `exclude=[]`:

In [ ]:
print(mne.pick_types(info, meg=False, eeg=True, exclude=[]))

Note that the `meg‍‍‍‍` and `fnirs` parameters of `pick_types()` accept strings as well as boolean values, to allow selecting only magnetom‍‍eter or‍‍ gradiometer channels (via `meg='mag'` or `meg='grad'`) or to pick only oxyhemoglobin or deoxyhemoglobin channels (via `fnirs='hbo'` or `fnirs='hbr'`, respectively).

A third way to pick channels from an `Info` object is to apply `regular expression` matching to the channel names using `mne.pick_channels_regexp()`. Here the `^` represents the beginning of the string and `.` character matches any single character, so both EEG and EOG channels will be selected:

In [ ]:
print(mne.pick_channels_regexp(info["ch_names"], "^E.G"))

`pick_channels_regexp()` can be especially useful for channels named according to the `10-20` system (e.g., to select all channels ending in “z” to get the midline, or all channels beginning with “O” to get the occipital channels). Note that `pick_channels_regexp()` uses the Python standard module `re` to perform regular expression matching; see the documentation of the `re` module for implementation details.

### Obtaining channel type information

Sometimes it can be useful to know channel type based on its index in the data array. For this case, use `mne.channel_type()`, which takes an `Info` object and a single integer channel index:

In [ ]:
print(mne.channel_type(info, 25))

To obtain several channel types at once, you could embed `channel_type()` in a `list comprehension`, or use the `get_channel_types()` method of a `Raw`, `Epochs`, or `Evoked` instance:

In [ ]:
picks = (25, 76, 77, 319)
print([mne.channel_type(info, x) for x in picks])
print(raw.get_channel_types(picks=picks))

Alternatively, you can get the indices of all channels of all channel types present in the data, using `channel_indices_by_type()`, which returns a `dict` with channel types as keys, and lists of channel indices as values:

In [ ]:
ch_idx_by_type = mne.channel_indices_by_type(info)
print(ch_idx_by_type.keys())
print(ch_idx_by_type["eog"])

### Dropping channels from an Info object

If you want to modify an `Info` object by eliminating some of the channels in it, you can use the `mne.pick_info()` function to pick the channels you want to keep and omit the rest:

In [ ]:
print(info["nchan"])
eeg_indices = mne.pick_types(info, meg=False, eeg=True)
print(mne.pick_info(info, eeg_indices)["nchan"])

We can also get a nice HTML representation in IPython like this:

In [ ]:
info

By default, `pick_info()` will make a copy of the original `Info` object before modifying it; if you want to modify it in-place, include the parameter `copy=False`.

## 1.5 Working with sensor locations

This tutorial describes how to read and plot sensor locations, and how MNE-Python handles physical locations of sensors.

### About montages and layouts

`Montages` contain sensor positions in 3D (x, y, z in meters), which can be assigned to existing EEG/MEG data. By specifying the locations of sensors relative to the brain, `Montages` play an important role in computing the forward solution and inverse estimates.

In contrast, `Layouts` are idealized 2D representations of sensor positions. They are primarily used for arranging individual sensor subplots in a topoplot or for showing the approximate relative arrangement of sensors as seen from above.

**Note:**
```
If you’re working with EEG data exclusively, you’ll want to use Montages, not layouts. Idealized montages (e.g., those provided by the manufacturer, or the ones shipping with MNE-Python mentioned below) are typically referred to as template montages.
```

### Working with built-in montages

The 3D coordinates of MEG sensors are included in the raw recordings from MEG systems. They are automatically stored in the `info` attribute of the `Raw` object upon loading. EEG electrode locations are much more variable because of differences in head shape. Idealized montages (”`template montages`”) for many EEG systems are included in MNE-Python, and you can get an overview of them by using `mne.channels.get_builtin_montages()`:

In [ ]:
builtin_montages = mne.channels.get_builtin_montages(descriptions=True)
for montage_name, montage_description in builtin_montages:
    print(f"{montage_name}: {montage_description}")

These built-in EEG montages can be loaded with `mne.channels.make_standard_montage`:

In [ ]:
easycap_montage = mne.channels.make_standard_montage("easycap-M1")
print(easycap_montage)

`Montage` objects have a `plot` method for visualizing the sensor locations in 2D or 3D:

In [ ]:
easycap_montage.plot()  # 2D
fig = easycap_montage.plot(kind="3d", show=False)  # 3D
fig = fig.gca().view_init(azim=70, elev=15)  # set view angle for tutorial

Once loaded, a montage can be applied to data with the `set_montage` method, for example `raw.set_montage()`, `epochs.set_montage()`, or `evoked.set_montage()`. This will only work with data whose EEG channel names correspond to those in the montage. (Therefore, we’re loading some EEG data below, and not the usual MNE “sample” dataset.)

You can then visualize the sensor locations via the `plot_sensors()` method.

It is also possible to skip the manual montage loading step by passing the montage name directly to the `set_montage()` method.

In [ ]:
ssvep_folder = mne.datasets.ssvep.data_path()
ssvep_data_raw_path = (
    ssvep_folder / "sub-02" / "ses-01" / "eeg" / "sub-02_ses-01_task-ssvep_eeg.vhdr"
)
ssvep_raw = mne.io.read_raw_brainvision(ssvep_data_raw_path, verbose=False)

# Use the preloaded montage
ssvep_raw.set_montage(easycap_montage)
fig = ssvep_raw.plot_sensors(show_names=True)

# Apply a template montage directly, without preloading
ssvep_raw.set_montage("easycap-M1")
fig = ssvep_raw.plot_sensors(show_names=True)

**Note:**

```
You may have noticed that the figures created via plot_sensors() contain fewer sensors than the result of easycap_montage.plot(). This is because the montage contains all channels defined for that EEG system; but not all recordings will necessarily use all possible channels. Thus when applying a montage to an actual EEG dataset, information about sensors that are not actually present in the data is removed.
```

### Plotting 2D sensor locations like EEGLAB

In MNE-Python, by default the head center is calculated using `fiducial points`. This means that the head circle represents the head circumference at the nasion and ear level, and not where it is commonly measured in the 10–20 EEG system (i.e., above the nasion at T4/T8, T3/T7, Oz, and Fpz).

If you prefer to draw the head circle using 10–20 conventions (which are also used by EEGLAB), you can pass `sphere='eeglab'`:

In [ ]:
fig = ssvep_raw.plot_sensors(show_names=True, sphere="eeglab")

Because the data we’re using here doesn’t contain an Fpz channel, its putative location was approximated automatically.

### Manually controlling 2D channel projection

Channel positions in 2D space are obtained by projecting their actual 3D positions onto a sphere, then projecting the sphere onto a plane. By default, a sphere with origin at `(0, 0, 0)` (x, y, z coordinates) and radius of `0.095` meters (9.5 cm) is used. You can use a different sphere radius by passing a single value as the sphere argument in any function that plots channels in 2D (like `plot` that we use here, but also for example `mne.viz.plot_topomap`):

In [ ]:
fig1 = easycap_montage.plot()  # default radius of 0.095
fig2 = easycap_montage.plot(sphere=0.07)

To change not only the radius, but also the sphere origin, pass a `(x, y, z, radius)` tuple as the `sphere` argument:

In [ ]:
fig = easycap_montage.plot(sphere=(0.03, 0.02, 0.01, 0.075))

### Reading sensor digitization files

In the sample data, the sensor positions are already available in the `info` attribute of the `Raw` object (see the documentation of the reading functions and `set_montage()` for details on how that works). Therefore, we can plot sensor locations directly from the `Raw` object using `plot_sensors()`, which provides similar functionality to `montage.plot()`. In addition, `plot_sensors()` supports channel selection by type, color-coding channels in various ways (by default, channels listed in `raw.info['bads']` will be plotted in red), and drawing in an existing Matplotlib `Axes` object (so the channel positions can easily be added as a subplot in a multi-panel figure):

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_path = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
sample_raw = mne.io.read_raw_fif(sample_data_raw_path, preload=False, verbose=False)

fig = plt.figure()
ax2d = fig.add_subplot(121)
ax3d = fig.add_subplot(122, projection="3d")
sample_raw.plot_sensors(ch_type="eeg", axes=ax2d)
sample_raw.plot_sensors(ch_type="eeg", axes=ax3d, kind="3d")
ax3d.view_init(azim=70, elev=15)

The previous 2D topomap reveals irregularities in the EEG sensor positions in the `sample dataset` — this is because the sensor positions in that dataset are digitizations of actual sensor positions on the head rather than idealized sensor positions based on a spherical head model. Depending on the digitization device (e.g., a Polhemus Fastrak digitizer), you need to use different montage reading functions (see `Supported formats for digitized 3D locations`). The resulting `montage` can then be added to `Raw` objects by passing it as an argument to the `set_montage()` method (just as we did before with the name of the predefined `'standard_1020'` montage). Once loaded, locations can be plotted with the `plot()` method and saved with the `save()` method of the `montage` object.

### Visualizing sensors in 3D surface renderings

It is also possible to render an image of an MEG sensor helmet using 3D surface rendering instead of matplotlib. This works by calling `mne.viz.plot_alignment()`:

In [ ]:
fig = mne.viz.plot_alignment(
    sample_raw.info,
    dig=False,
    eeg=False,
    surfaces=[],
    meg=["helmet", "sensors"],
    coord_frame="meg",
)
mne.viz.set_3d_view(fig, azimuth=50, elevation=90, distance=0.5)

Note that `plot_alignment()` requires an `Info` object, and can also render MRI surfaces of the scalp, skull, and brain (by passing a dict with keys like `'head'`, `'outer_skull'` or `'brain'` to the surfaces parameter). This makes the function useful for `assessing coordinate frame transformations`. For examples of various uses of `plot_alignment()`, see `Plotting sensor layouts of EEG systems, Plotting EEG sensors on the scalp`, and `Plotting sensor layouts of MEG systems`.

### Working with layout files

Similar to montages, many layout files are included with MNE-Python. They are stored in the `mne/channels/data/layouts` folder:

In [ ]:
layout_dir = Path(mne.__file__).parent / "channels" / "data" / "layouts"
layouts = sorted(path.name for path in layout_dir.iterdir())
print("\n" "BUILT-IN LAYOUTS\n" "================")
print("\n".join(layouts))

To load a layout file, use the `mne.channels.read_layout` function. You can then visualize the layout using its `plot` method:

In [ ]:
biosemi_layout = mne.channels.read_layout("biosemi")

Similar to the `picks` argument for selecting channels from `Raw` objects, the `plot()` method of `Layout` objects also has a `picks` argument. However, because layouts only contain information about sensor name and location (not sensor type), the `plot()` method only supports picking channels by index (not by name or by type). In the following example, we find the desired indices using `numpy.where()`; selection by name or type is possible with `mne.pick_channels()` or `mne.pick_types()`.

In [ ]:
midline = np.where([name.endswith("z") for name in biosemi_layout.names])[0]
biosemi_layout.plot(picks=midline)

If you have a `Raw` object that contains sensor positions, you can create a `Layout` object with either `mne.channels.make_eeg_layout()` or `mne.channels.find_layout()`.

In [ ]:
layout_from_raw = mne.channels.make_eeg_layout(sample_raw.info)
# same result as mne.channels.find_layout(raw.info, ch_type='eeg')
layout_from_raw.plot()

**Note:**

```
There is no corresponding make_meg_layout() function because sensor locations are fixed in an MEG system (unlike in EEG, where sensor caps deform to fit snugly on a specific head). Therefore, MEG layouts are consistent (constant) for a given system and you can simply load them with mne.channels.read_layout() or use mne.channels.find_layout() with the ch_type parameter (as previously demonstrated for EEG).
```

All `Layout` objects have a `save` method that writes layouts to disk as either `.lout` or `.lay` formats (inferred from the file extension contained in the `fname` argument). The choice between `.lout` and `.lay` format only matters if you need to load the layout file in some other application (MNE-Python can read both formats).

## 1.6 Configuring MNE-Python

This tutorial covers how to configure MNE-Python to suit your local system and your analysis preferences.

### Getting and setting configuration variables

Configuration variables are read and written using the functions `mne.get_config()` and `mne.set_config()`. To read a specific configuration variable, pass its name to `get_config()` as the `key` parameter (`key` is the first parameter so you can pass it unnamed if you want):

In [ ]:
print(mne.get_config("MNE_USE_CUDA"))
print(type(mne.get_config("MNE_USE_CUDA")))

Note that the string values read from the JSON file are not parsed in any way, so `get_config()` returns a string even for true/false config values, rather than a Python `boolean`. Similarly, `set_config()` will only set string values (or `None` values, to unset a variable):

In [ ]:
try:
    mne.set_config("MNE_USE_CUDA", True)
except TypeError as err:
    print(err)

If you’re unsure whether a config variable has been set, there is a convenient way to check it and provide a fallback in case it doesn’t exist: `get_config()` has a `default` parameter.

In [ ]:
print(mne.get_config("missing_config_key", default="fallback value"))

There are also two convenience modes of `get_config()`. The first will return a `dict` containing all config variables (and their values) that have been set on your system; this is done by passing `key=None` (which is the default, so it can be omitted):

In [ ]:
print(mne.get_config())  # same as mne.get_config(key=None)

The second convenience mode will return a `tuple` of all the keys that MNE-Python recognizes and uses, regardless of whether they’ve been set on your system. This is done by passing an empty string `''` as the `key`:

In [ ]:
print(mne.get_config(key=""))

It is possible to add config variables that are not part of the recognized list, by passing any arbitrary key to `set_config()`. This will yield a warning, however, which is a nice check in cases where you meant to set a valid key but simply misspelled it:

In [ ]:
mne.set_config("MNEE_USE_CUUDAA", "false")

Let’s delete that config variable we just created. To unset a config variable, use `set_config()` with `value=None`. Since we’re still dealing with an unrecognized key (as far as MNE-Python is concerned) we’ll still get a warning, but the key will be unset:

In [ ]:
mne.set_config("MNEE_USE_CUUDAA", None)
assert "MNEE_USE_CUUDAA" not in mne.get_config("")

### Where configurations are stored

MNE-Python stores configuration variables in a `JSON` file. By default, this file is located in `%USERPROFILE%\.mne\mne-python.json` on Windows and `$HOME/.mne/mne-python.json` on Linux or macOS. You can get the full path to the config file with `mne.get_config_path()`.

In [ ]:
print(mne.get_config_path())

However it is not a good idea to directly edit files in the .mne directory; use the getting and setting functions described in `the previous section`.

If for some reason you want to load the configuration from a different location, you can pass the `home_dir` parameter to `get_config_path()`, specifying the parent directory of the `.mne` directory where the configuration file you wish to load is stored.

### Using environment variables

For compatibility with `MNE-C`, MNE-Python also reads and writes `environment variables` to specify configuration. This is done with the same functions that read and write the JSON configuration, and is controlled with the parameters `use_env` and `set_env`. By default, `get_config()` will check `os.environ` before checking the MNE-Python JSON file; to check only the JSON file use `use_env=False`. To demonstrate, here’s an environment variable that is not specific to MNE-Python (and thus is not in the JSON config file):

In [ ]:
# make sure it's not in the JSON file (no error means our assertion held):
assert mne.get_config("PATH", use_env=False) is None
# but it *is* in the environment:
print(mne.get_config("PATH"))

Also by default, `set_config()` will set values in both the JSON file and in `os.environ`; to set a config variable only in the JSON file use `set_env=False`. Here we’ll use `print()` statement to confirm that an environment variable is being created and deleted (we could have used the Python `assert statement` instead, but it doesn’t print any output when it succeeds so it’s a little less obvious):

In [ ]:
mne.set_config("foo", "bar", set_env=False)
print("foo" in os.environ.keys())
mne.set_config("foo", "bar")
print("foo" in os.environ.keys())
mne.set_config("foo", None)  # unsetting a key deletes var from environment
print("foo" in os.environ.keys())

## 1.7 Getting started with mne.Report

`mne.Report` is a way to create interactive HTML summaries of your data. These reports can show many different visualizations for one or multiple participants. A common use case is creating diagnostic summaries to check data quality at different stages in the processing pipeline. The report can show things like plots of data before and after each preprocessing step, epoch rejection statistics, MRI slices with overlaid BEM shells, all the way up to plots of estimated cortical activity.

Compared to a Jupyter notebook, `mne.Report` is easier to deploy, as the HTML pages it generates are self-contained and do not require a running Python environment. However, it is less flexible as you can't change code and re-run something directly within the browser. This tutorial covers the basics of building a report.

In [ ]:
data_path = Path(mne.datasets.sample.data_path(verbose=False))
sample_dir = data_path / "MEG" / "sample"
subjects_dir = data_path / "subjects"

Before getting started with `mne.Report`, make sure the files you want
to render follow the filename conventions defined by MNE:


Data object                        | Filename convention (ends with)
---------------------------------- | ---------------------------------------
`~mne.io.Raw`                      | ``-raw.fif(.gz)``, ``-raw_sss.fif(.gz)``,``-raw_tsss.fif(.gz)``,    ``_eeg.fif(.gz)``, ``_ieeg.fif(.gz)``

                                    
                                    
events                             | ``-eve.fif(.gz)``
---------------------------------- | ----------------------
`~mne.Epochs`                      | ``-epo.fif(.gz)``
`~mne.Evoked`                      |  ``-ave.fif(.gz)``
`~mne.Covariance`                  |``-cov.fif(.gz)``
`~mne.Projection`                  |``-proj.fif(.gz)``
`~mne.transforms.Transform`        |``-trans.fif(.gz)``
`~mne.Forward`                     |``-fwd.fif(.gz)``
`~mne.minimum_norm.InverseOperator`|``-inv.fif(.gz)``


Alternatively, the dash ``-`` in the filename may be replaced with an underscore ``_``.

The basic process for creating an HTML report is to instantiate the `Report` class and then use one or more of its many methods to add content, one element at a time.

You may also use the `parse_folder()` method to select particular files to include in the report. But more on that later.


### Adding `Raw` data

Raw data can be added via the `mne.Report.add_raw()` method. It can operate with a path to a raw file and `Raw` objects, and will produce – among other output – a slider that allows you to scrub through 10 equally-spaced 1-second segments of the data:

**Warning**

```
In the following example, we crop the raw data to 60 seconds merely to speed up processing; this is not usually recommended!
```

In [ ]:
raw_path = sample_dir / "sample_audvis_filt-0-40_raw.fif"
raw = mne.io.read_raw(raw_path)
raw.pick(picks=["eeg", "eog", "stim"]).crop(tmax=60).load_data()

report = mne.Report(title="Raw example")
# This method also accepts a path, e.g., raw=raw_path
report.add_raw(raw=raw, title="Raw", psd=False)  # omit PSD plot
report.save("report_raw.html", overwrite=True)

### Adding events

Events can be added via `mne.Report.add_events`. You also need to supply the sampling frequency used during the recording; this information is used to generate a meaningful time axis.

In [ ]:
events_path = sample_dir / "sample_audvis_filt-0-40_raw-eve.fif"
events = mne.find_events(raw=raw)
sfreq = raw.info["sfreq"]

report = mne.Report(title="Events example")
report.add_events(events=events_path, title="Events from Path", sfreq=sfreq)
report.add_events(events=events, title='Events from "events"', sfreq=sfreq)
report.save("report_events.html", overwrite=True)

### Adding Epochs

Epochs can be added via `mne.Report.add_epochs()`. Note that although this method accepts a path to an epochs file too, in the following example we only add epochs that we create on the fly from raw data. To demonstrate the representation of epochs metadata, we’ll add some of that too.

In [ ]:
event_id = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
    "face": 5,
    "buttonpress": 32,
}

metadata, _, _ = mne.epochs.make_metadata(
    events=events, event_id=event_id, tmin=-0.2, tmax=0.5, sfreq=raw.info["sfreq"]
)
epochs = mne.Epochs(raw=raw, events=events, event_id=event_id, metadata=metadata)

report = mne.Report(title="Epochs example")
report.add_epochs(epochs=epochs, title='Epochs from "epochs"')
report.save("report_epochs.html", overwrite=True)

### Adding Evoked

Evoked data can be added via `mne.Report.add_evokeds()`. By default, the `Evoked.comment` attribute of each evoked will be used as a title. We can specify custom titles via the `titles` parameter. Again, this method also accepts the path to an evoked file stored on disk; in the following example, however, we load the evokeds manually first, since we only want to add a subset of them to the report. The evokeds are not baseline-corrected, so we apply baseline correction, too. Lastly, by providing an (optional) noise covariance, we can add plots evokeds that were “whitened” using this covariance matrix.

By default, this method will produce snapshots at 21 equally-spaced time points (or fewer, if the data contains fewer time points). We can adjust this via the `n_time_points` parameter.

In [ ]:
evoked_path = sample_dir / "sample_audvis-ave.fif"
cov_path = sample_dir / "sample_audvis-cov.fif"

evokeds = mne.read_evokeds(evoked_path, baseline=(None, 0))
evokeds_subset = evokeds[:2]  # The first two
for evoked in evokeds_subset:
    evoked.pick("eeg")  # just for speed of plotting

report = mne.Report(title="Evoked example")
report.add_evokeds(
    evokeds=evokeds_subset,
    titles=["evoked 1", "evoked 2"],  # Manually specify titles
    noise_cov=cov_path,
    n_time_points=5,
)
report.save("report_evoked.html", overwrite=True)

### Adding Covariance

(Noise) covariance objects can be added via `mne.Report.add_covariance()`. The method accepts `Covariance` objects and the path to a file on disk. It also expects us to pass an Info object or the path to a file to read the measurement `info` from, as well as a title.

In [ ]:
cov_path = sample_dir / "sample_audvis-cov.fif"

report = mne.Report(title="Covariance example")
report.add_covariance(cov=cov_path, info=raw_path, title="Covariance")
report.save("report_cov.html", overwrite=True)

### Adding `Projection` vectors

`Projection` vectors can be added via `mne.Report.add_projs()`. The method requires an `Info` object (or the path to one) and a title. Projectors found in the `Info` will be visualized. You may also supply a list of `Projection` objects or a path to projectors stored on disk. In this case, the channel information is read from the `Info`, but projectors potentially included will be ignored; instead, only the explicitly passed projectors will be plotted.

In [ ]:
ecg_proj_path = sample_dir / "sample_audvis_ecg-proj.fif"
report = mne.Report(title="Projectors example")
report.add_projs(info=raw_path, title="Projs from info")
report.add_projs(info=raw_path, projs=ecg_proj_path, title="ECG projs from path")
report.save("report_projs.html", overwrite=True)

### Adding ICA

`ICA` objects can be added via `mne.Report.add_ica()`. Aside from the parameters `ica` (that accepts an ICA instance or a path to an `ICA` object stored on disk) and the `title`, there is a third required parameter, `inst`. `inst` is used to specify a `Raw` or `Epochs` object for producing ICA property plots and overlay plots demonstrating the effects of ICA cleaning. If, instead, you only want to generate ICA component topography plots, explicitly pass `inst=None`.

**Note**: `mne.Report.add_ica()` only works with fitted ICAs.

You can optionally specify for which components to produce topography and properties plots by passing `picks`. By default, all components will be shown. It is also possible to pass evoked signals based on ECG and EOG events via `ecg_evoked` and `eog_evoked`. This allows you directly see the effects of ICA component removal on these artifactual signals. Artifact detection scores produced by `find_bads_ecg()` and `find_bads_eog()` can be passed via the `ecg_scores` and `eog_scores` parameters, respectively, producing visualizations of the scores for each ICA component.

Lastly, by passing `n_jobs`, you may largely speed up the generation of the properties plots by enabling parallel execution.

**Warning**:
```
In the following example, we request a small number of ICA components to estimate, set the threshold for assuming ICA convergence to a very liberal value, and only visualize 2 of the components. All of this is done to largely reduce the processing time of this tutorial, and is usually not recommended for an actual data analysis.
```

In [ ]:
ica = mne.preprocessing.ICA(
    n_components=5,  # fit 5 ICA components
    fit_params=dict(tol=0.01),  # assume very early on that ICA has converged
)

ica.fit(inst=raw)

# create epochs based on EOG events, find EOG artifacts in the data via pattern
# matching, and exclude the EOG-related ICA components
eog_epochs = mne.preprocessing.create_eog_epochs(raw=raw)
eog_components, eog_scores = ica.find_bads_eog(
    inst=eog_epochs,
    ch_name="EEG 001",  # a channel close to the eye
    threshold=1,  # lower than the default threshold
)
ica.exclude = eog_components

report = mne.Report(title="ICA example")
report.add_ica(
    ica=ica,
    title="ICA cleaning",
    picks=ica.exclude,  # plot the excluded EOG components
    inst=raw,
    eog_evoked=eog_epochs.average(),
    eog_scores=eog_scores,
    n_jobs=None,  # could be increased!
)
report.save("report_ica.html", overwrite=True)

### Adding MRI with BEM

MRI slices with superimposed traces of the boundary element model (BEM) surfaces can be added via `mne.Report.add_bem()`. All you need to pass is the FreeSurfer subject name and subjects directory, and a title. To reduce the resulting file size, you may pass the `decim` parameter to only include every n-th volume slice, and `width` to specify the width of the resulting figures in pixels.

In [ ]:
report = mne.Report(title="BEM example")
report.add_bem(
    subject="sample",
    subjects_dir=subjects_dir,
    title="MRI & BEM",
    decim=40,
    width=256,
)
report.save("report_mri_and_bem.html", overwrite=True)

### Adding coregistration

The sensor alignment (`head -> mri` transformation obtained by “coregistration”) can be visualized via `mne.Report.add_trans()`. The method expects the transformation either as a `Transform` object or as a path to a `trans.fif` file, the FreeSurfer subject name and subjects directory, and a title. The `alpha` parameter can be used to control the transparency of the head, where a value of 1 means fully opaque.

In [ ]:
trans_path = sample_dir / "sample_audvis_raw-trans.fif"

report = mne.Report(title="Coregistration example")
report.add_trans(
    trans=trans_path,
    info=raw_path,
    subject="sample",
    subjects_dir=subjects_dir,
    alpha=1.0,
    title="Coregistration",
)
report.save("report_coregistration.html", overwrite=True)

### Adding a Forward solution

Forward solutions (“leadfields”) can be added by passing a `Forward` object or the path to a forward solution stored on disk to `meth:mne.Report.add_forward`.

In [ ]:
fwd_path = sample_dir / "sample_audvis-meg-oct-6-fwd.fif"

report = mne.Report(title="Forward solution example")
report.add_forward(forward=fwd_path, title="Forward solution")
report.save("report_forward_sol.html", overwrite=True)

### Adding an InverseOperator

An inverse operator can be added via `mne.Report.add_inverse_operator()`. The method expects an `InverseOperator` object or a path to one stored on disk, and a title.

In [ ]:
inverse_op_path = sample_dir / "sample_audvis-meg-oct-6-meg-inv.fif"

report = mne.Report(title="Inverse operator example")
report.add_inverse_operator(inverse_operator=inverse_op_path, title="Inverse operator")
report.save("report_inverse_op.html", overwrite=True)

### Adding a `SourceEstimate`

An inverse solution (also called source estimate or source time course, STC) can be added via `mne.Report.add_stc()`. The method expects an `SourceEstimate`, the corresponding FreeSurfer subject name and subjects directory, and a title. By default, it will produce snapshots at 51 equally-spaced time points (or fewer, if the data contains fewer time points). We can adjust this via the `n_time_points` parameter.

In [ ]:
stc_path = sample_dir / "sample_audvis-meg"

report = mne.Report(title="Source estimate example")
report.add_stc(
    stc=stc_path,
    subject="sample",
    subjects_dir=subjects_dir,
    title="Source estimate",
    n_time_points=2,  # few for speed
)
report.save("report_inverse_sol.html", overwrite=True)

### Adding source code (e.g., a Python script)

It is possible to add code or scripts (e.g., the scripts you used for analysis) to the report via `mne.Report.add_code()`. The code blocks will be automatically syntax-highlighted. You may pass a string with the respective code snippet, or the path to a file. If you pass a path, it **must** be a `pathlib.Path` object (and not a string), otherwise it will be treated as a code literal.

Optionally, you can specify which programming language to assume for syntax highlighting by passing the `language` parameter. By default, we’ll assume the provided code is Python.

In [ ]:
mne_init_py_path = Path(mne.__file__)  # __init__.py in the MNE-Python root
mne_init_py_content = mne_init_py_path.read_text(encoding="utf-8")

report = mne.Report(title="Code example")
report.add_code(code=mne_init_py_path, title="Code from Path")
report.add_code(code=mne_init_py_content, title="Code from string")

report.save("report_code.html", overwrite=True)

### Adding custom figures

Custom Matplotlib figures can be added via `add_figure()`. Required parameters are the figure and a title. Optionally, may add a caption to appear below the figure. You can also specify the image format of the image file that will be generated from the figure, so it can be embedded in the HTML report.

In [ ]:
x = np.linspace(start=0, stop=10, num=100)
y = x**2

fig, ax = plt.subplots()
ax.plot(x, y, ls="--", lw=2, color="blue", label="my function")
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.legend()

report = mne.Report(title="Figure example")
report.add_figure(
    fig=fig,
    title="A custom figure",
    caption="A blue dashed line reaches up into the sky …",
    image_format="PNG",
)
report.save("report_custom_figure.html", overwrite=True)
plt.close(fig)

Multiple figures can be grouped into a single section via the `section` parameter.

In [ ]:
fig_1, ax_1 = plt.subplots()
ax_1.plot([1, 2, 3])

fig_2, ax_2 = plt.subplots()
ax_2.plot([3, 2, 1])

section = "Section example"

report = mne.Report(title="Figure section example")
report.add_figure(fig=fig_1, title="Figure 1", section=section, tags="fig-1")
report.add_figure(fig=fig_2, title="Figure 2", section=section, tags="fig-2")
report.save("report_custom_figure_sections.html", overwrite=True)
plt.close(fig_1)
plt.close(fig_2)

The `mne.Report.add_figure()` method can also add multiple figures at once. In this case, a slider will appear, allowing users to intuitively browse the figures. To make this work, you need to provide a collection o figures, a title, and optionally a collection of captions.

In the following example, we will read the MNE logo as a Matplotlib figure and rotate it with different angles. Each rotated figure and its respective caption will be added to a list, which is then used to create the slider.

In [ ]:
mne_logo_path = Path(mne.__file__).parent / "icons" / "mne_icon-cropped.png"
fig_array = plt.imread(mne_logo_path)
rotation_angles = np.linspace(start=0, stop=360, num=8, endpoint=False)

figs = []
captions = []
for angle in rotation_angles:
    # Rotate and remove some rounding errors to avoid Matplotlib warnings
    fig_array_rotated = scipy.ndimage.rotate(input=fig_array, angle=angle)
    fig_array_rotated = fig_array_rotated.clip(min=0, max=1)

    # Create the figure
    fig, ax = plt.subplots(figsize=(3, 3), layout="constrained")
    ax.imshow(fig_array_rotated)
    ax.set_axis_off()

    # Store figure and caption
    figs.append(fig)
    captions.append(f"Rotation angle: {round(angle, 1)}°")

report = mne.Report(title="Multiple figures example")
report.add_figure(fig=figs, title="Fun with figures! 🥳", caption=captions)
report.save("report_custom_figures.html", overwrite=True)
for fig in figs:
    plt.close(fig)
del figs

### Adding image files

Existing images (e.g., photos, screenshots, sketches etc.) can be added to the report via `mne.Report.add_image()`. Supported image formats include JPEG, PNG, GIF, and SVG (and possibly others). Like with Matplotlib figures, you can specify a caption to appear below the image.

In [ ]:
report = mne.Report(title="Image example")
report.add_image(
    image=mne_logo_path, title="MNE", caption="Powered by 🧠 🧠 🧠 around the world!"
)
report.save("report_custom_image.html", overwrite=True)

### Working with tags

Each `add_*` method accepts a keyword parameter `tags`, which can be used to pass one or more tags to associate with the respective content elements. By default, each `add_*` method adds a tag describing the data type, e.g., `evoked` or `source-estimate`. When viewing the HTML report, the `Filter by tags` dropdown menu can be used to interactively show or hide content with specific tags. This allows you e.g. to only view `evoked` or `participant-001` data, should you have added those tags. Visible tags will appear with blue, and hidden tags with gray background color.

To toggle the visibility of all tags, use the respective checkbox in the `Filter by tags` dropdown menu, or press `T`.

In [ ]:
report = mne.Report(title="Tags example")
report.add_image(
    image=mne_logo_path,
    title="MNE Logo",
    tags=("image", "mne", "logo", "open-source"),
)
report.save("report_tags.html", overwrite=True)

### Editing a saved report

Saving to HTML is a write-only operation, meaning that we cannot read an `.html` file back as a `Report` object. In order to be able to edit a report once it’s no longer in-memory in an active Python session, save it as an HDF5 file instead of HTML:

In [ ]:
report = mne.Report(title="Saved report example", verbose=True)
report.add_image(image=mne_logo_path, title="MNE 1")
report.save("report_partial.hdf5", overwrite=True)

The saved report can be read back and modified or amended. This allows the possibility to e.g. run multiple scripts in a processing pipeline, where each script adds new content to an existing report.

In [ ]:
report_from_disk = mne.open_report("report_partial.hdf5")
report_from_disk.add_image(image=mne_logo_path, title="MNE 2")
report_from_disk.save("report_partial.hdf5", overwrite=True)

To make this even easier, `mne.Report` can be used as a context manager (note the `with` statement)`):

In [ ]:
with mne.open_report("report_partial.hdf5") as report:
    report.add_image(image=mne_logo_path, title="MNE 3")
    report.save("report_final.html", overwrite=True)

### Adding an entire folder of files

We also provide a way to add an entire folder of files to the report at once, without having to invoke the individual `add_*` methods outlined above for each file. This approach, while convenient, provides less flexibility with respect to content ordering, tags, titles, etc.

For our first example, we’ll generate a barebones report for all the `.fif` files containing raw data in the sample dataset, by passing the pattern `*raw.fif` to `parse_folder()`. We’ll omit the `subject` and `subjects_dir` parameters from the `Report` constructor, but we’ll also pass `render_bem=False` to the `parse_folder()` method — otherwise we would get a warning about not being able to render MRI and `trans` files without knowing the subject. To save some processing time in this tutorial, we’re also going to disable rendering of the butterfly plots for the `Raw` data by passing `raw_butterfly=False`.

Which files are included depends on both the `pattern` parameter passed to `parse_folder()` and also the `subject` and `subjects_dir` parameters provided to the `Report` constructor.

In [ ]:
report = mne.Report(title="parse_folder example")
report.parse_folder(
    data_path=data_path, pattern="*raw.fif", render_bem=False, raw_butterfly=False
)
report.save("report_parse_folder_basic.html", overwrite=True)

By default, the power spectral density and SSP projectors of the `Raw` files are not shown to speed up report generation. You can add them by passing `raw_psd=True` and `projs=True` to the `Report` constructor. Like in the previous example, we’re going to omit the butterfly plots by passing `raw_butterfly=False`. Lastly, let’s also refine our pattern to select only the filtered raw recording (omitting the unfiltered data and the empty-room noise recordings).

In [ ]:
pattern = "sample_audvis_filt-0-40_raw.fif"
report = mne.Report(title="parse_folder example 2", raw_psd=True, projs=True)
report.parse_folder(
    data_path=data_path, pattern=pattern, render_bem=False, raw_butterfly=False
)
report.save("report_parse_folder_raw_psd_projs.html", overwrite=True)

This time we’ll pass a specific `subject` and `subjects_dir` (even though there’s only one subject in the sample dataset) and remove our `render_bem=False` parameter so we can see the MRI slices, with BEM contours overlaid on top if available. Since this is computationally expensive, we’ll also pass the `mri_decim` parameter for the benefit of our documentation servers, and skip processing the `.fif` files.

In [ ]:
report = mne.Report(
    title="parse_folder example 3", subject="sample", subjects_dir=subjects_dir
)
report.parse_folder(data_path=data_path, pattern="", mri_decim=40)
report.save("report_parse_folder_mri_bem.html", overwrite=True)

Now let’s look at how `Report` handles `Evoked` data (we will skip the MRIs to save computation time).

The MNE sample dataset we’re using in this example has not been baseline-corrected; so let’s apply baseline correction this now for the report!

To request baseline correction, pass a `baseline` argument to `Report`, which should be a tuple with the starting and ending time of the baseline period. For more details, see the documentation on `apply_baseline`. Here, we will apply baseline correction for a baseline period from the beginning of the time interval to time point zero.

Lastly, we want to render the “whitened” evoked data, too. Whitening requires us to specify the path to a covariance matrix file via the `cov_fname` parameter of `Report`.

Now, let’s put all of this together! Here we use a temporary directory for speed so we can render a single Evoked instance, using just EEG channels.

In [ ]:
baseline = (None, 0)
cov_fname = sample_dir / "sample_audvis-cov.fif"
pattern = "sample_audvis-ave.fif"
evoked = mne.read_evokeds(sample_dir / pattern)[0].pick("eeg").decimate(4)
report = mne.Report(
    title="parse_folder example 4", baseline=baseline, cov_fname=cov_fname
)
with tempfile.TemporaryDirectory() as path:
    evoked.save(Path(path) / pattern)
    report.parse_folder(
        path, pattern=pattern, render_bem=False, n_time_points_evokeds=5
    )
report.save("report_parse_folder_evoked.html", overwrite=True)

### Adding custom HTML (e.g., a description text)

The `add_html()` method allows you to add custom HTML to your report. This feature can be very convenient to add short descriptions, lists, or reminders to your report (among many other things you can think of encoding in HTML).

In [ ]:
report = mne.Report(title="Report on hypothesis 1")

my_html = """
<p>We have the following hypothesis:</p>
<ol>
<li>There is a difference between images showing man-made vs. natural
environments</li>
<li>This difference manifests itself most strongly in the amplitude of the
N1 ERP component</li>
</ol>
<p>Below we show several plots and tests of the data.</p>
"""

report.add_html(title="Hypothesis", html=my_html)
report.save("report_add_html.html", overwrite=True)